In [9]:
import numpy as np
import pandas as pd


In [10]:
from dataset import load_dataset, load_labels, convert_to_epochs, load_channels
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features
from classifiers import LR, KNN, SVM, NN
import variables as v

In [11]:
#Filtered from SAM40
dataset_ = load_dataset(data_type="filtered", test_type="Arithmetic")
dataset = convert_to_epochs(dataset_, v.NUM_CHANNELS, v.SFREQ)

#ICA filtered
dataset_ica_ = load_dataset(data_type="ica1", test_type="Arithmetic")
dataset_ica = convert_to_epochs(dataset_ica_, v.NUM_CHANNELS, v.SFREQ)

#ICA filtered two times
dataset_ica_2_ = load_dataset(data_type="ica2", test_type="Arithmetic")
dataset_ica_2 = convert_to_epochs(dataset_ica_2_, v.NUM_CHANNELS, v.SFREQ)

In [12]:
features = time_series_features(dataset, v.NUM_CHANNELS)
features_ica = time_series_features(dataset_ica, v.NUM_CHANNELS)
features_ica_2 = time_series_features(dataset_ica_2, v.NUM_CHANNELS)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features
data_ica = features_ica
data_ica_2 = features_ica_2

(120, 25, 32, 128)
(120, 25, 32, 128)
(120, 25, 32, 128)


In [13]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

### Linear Regression

In [7]:
print('Filtered data')
LR(data,label)
print('\nICA filtered data')
LR(data_ica,label)
print('\nICA filtered data round 2')
LR(data_ica_2,label)

Filtered data
              precision    recall  f1-score   support

       False       0.71      0.69      0.70       527
        True       0.66      0.67      0.66       463

    accuracy                           0.68       990
   macro avg       0.68      0.68      0.68       990
weighted avg       0.68      0.68      0.68       990


 Confusion matrix:
[[364 163]
 [152 311]]

 Sensitivity: 0.7054263565891473

 Specificity: 0.6561181434599156

ICA filtered data
              precision    recall  f1-score   support

       False       0.62      0.60      0.61       527
        True       0.56      0.58      0.57       463

    accuracy                           0.59       990
   macro avg       0.59      0.59      0.59       990
weighted avg       0.59      0.59      0.59       990


 Confusion matrix:
[[314 213]
 [193 270]]

 Sensitivity: 0.6193293885601578

 Specificity: 0.5590062111801242

ICA filtered data round 2
              precision    recall  f1-score   support

       Fa

### KNN

In [7]:
print('Filtered data')
KNN(data,label)
print('\nICA filtered data')
KNN(data_ica,label)
print('\nICA filtered data round 2')
KNN(data_ica_2,label)

Filtered data
              precision    recall  f1-score   support

       False       0.79      0.77      0.78       311
        True       0.76      0.78      0.77       289

    accuracy                           0.78       600
   macro avg       0.77      0.78      0.77       600
weighted avg       0.78      0.78      0.78       600


 Confusion matrix:
[[241  70]
 [ 65 224]]

 Sensitivity: 0.7875816993464052

 Specificity: 0.7619047619047619

ICA filtered data
              precision    recall  f1-score   support

       False       0.63      0.65      0.64       311
        True       0.61      0.59      0.60       289

    accuracy                           0.62       600
   macro avg       0.62      0.62      0.62       600
weighted avg       0.62      0.62      0.62       600


 Confusion matrix:
[[202 109]
 [118 171]]

 Sensitivity: 0.63125

 Specificity: 0.6107142857142858

ICA filtered data round 2
              precision    recall  f1-score   support

       False       0

### SVM

In [8]:
print('Filtered data')
SVM(data,label)
print('\nICA filtered data')
SVM(data_ica,label)
print('\nICA filtered data round 2')
SVM(data_ica_2,label)

Filtered data
              precision    recall  f1-score   support

       False       0.97      0.95      0.96       311
        True       0.95      0.97      0.96       289

    accuracy                           0.96       600
   macro avg       0.96      0.96      0.96       600
weighted avg       0.96      0.96      0.96       600


 Confusion matrix:
[[296  15]
 [ 10 279]]

 Sensitivity: 0.9673202614379085

 Specificity: 0.9489795918367347

ICA filtered data
              precision    recall  f1-score   support

       False       0.70      0.84      0.76       311
        True       0.78      0.61      0.68       289

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.72       600
weighted avg       0.74      0.73      0.73       600


 Confusion matrix:
[[262  49]
 [113 176]]

 Sensitivity: 0.6986666666666667

 Specificity: 0.7822222222222223

ICA filtered data round 2
              precision    recall  f1-score   support

       Fa

# Neural Network

In [15]:
print('Filtered data')
NN(data,label)


Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.8391666412353516

Best val_accuracy So Far: 0.8650000095367432
Total elapsed time: 00h 01m 38s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       311
           1       0.89      0.84      0.86       289

    accuracy                           0.87       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.87      0.87       600

[[281  30]
 [ 47 242]]


In [16]:
print('\nICA filtered data')
NN(data_ica,label)


Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.6625000238418579

Best val_accuracy So Far: 0.6658333539962769
Total elapsed time: 00h 01m 30s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 887us/step
              precision    recall  f1-score   support

           0       0.65      0.69      0.67       311
           1       0.65      0.61      0.62       289

    accuracy                           0.65       600
   macro avg       0.65      0.65      0.65       600
weighted avg       0.65      0.65      0.65       600

[[215  96]
 [114 175]]


In [17]:
print('\nICA filtered data round 2')
NN(data_ica_2,label)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.7199999988079071

Best val_accuracy So Far: 0.8600000143051147
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       311
           1       0.93      0.84      0.88       289

    accuracy                           0.89       600
   macro avg       0.90      0.89      0.89       600
weighted avg       0.90      0.89      0.89       600

[[293  18]
 [ 46 243]]
